# Adequacy - Human Impact 4.3

### Import libraries and set up environment

In [1]:
import os
import sys

import pandas as pd
print(pd.__version__)

import numpy as np
print(np.__version__)

import arcpy

import arcgis
print(arcgis.__version__)

from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer

1.4.4
1.20.1
2.1.0.2


In [2]:
# Set the workspace and environment settings

arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
arcpy.env.overwriteOutput = True 

#### 4.3 Human Impact

In [3]:
# Add the cumulative_impact_2013.tif layer to the project (from https://knb.ecoinformatics.org/view/doi:10.5063/F12B8WBS)

output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.RasterToGeodatabase(
    Input_Rasters=[r"C:\Users\bexg\Downloads\cumulative_impact_2013.tif"],
    Output_Geodatabase=output_gdb,
    Configuration_Keyword=""
)

<Result ''>

In [5]:
# Clip the cumulative impact data to the study area

in_feature = "cumulative_impact_2013"
mask_feature = "Caribbean_detailed_subregions_mol"
out_feature = "\\Impact_Carib_Adq4_3"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"


out_raster = arcpy.sa.ExtractByMask(
    in_raster= in_feature,
    in_mask_data= mask_feature,
    extraction_area="INSIDE",
    analysis_extent='-9355915.704 501761.6975 -4929940.5223 3747456.4946 PROJCS["WGS_1984_Mollweide",GEOGCS["GCS_WGS_1984",DATUM["D_unknown",SPHEROID["WGS84",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["false_easting",0.0],PARAMETER["false_northing",0.0],PARAMETER["central_meridian",0.0],UNIT["Meter",1.0]]'
)
out_raster.save(output_gdb + out_feature)

In [ ]:
# Reclassify the data into quartiles, such that the 25% most impacted pixels are given a value of 2
# and the other 75% of locations (lower impacted locations) were given a value of 1
# Note that it's easiest to do this in the ArcGIS GUI to automatically classify the values in quantiles (with 4 classes)

in_raster = "Impact_Carib_Adq4_3"
out_feature = "\\Impact_Carib_ReclassQuart_Adq4_3"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

out_raster = arcpy.sa.Reclassify(
    in_raster= in_raster,
    reclass_field="VALUE",
    remap="0.134819 1.163432 1;1.163432 1.477731 1;1.477731 1.792029 1;1.792029 7.420828 2",
    missing_values="DATA"
)
out_raster.save(output_gdb + out_feature)

In [27]:
# Assign each subregion in the study area a unique two-digit identification number

# First make a copy of the subregions layer
in_feature = "Caribbean_detailed_subregions_mol"
out_feature = "\\Caribbean_detailed_subregions_mol_HumanImpactIDs"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.ExportFeatures(
    in_features= in_feature,
    out_features=output_gdb + out_feature,
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
)

# Define the inputs
arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"
in_feature = "Caribbean_detailed_subregions_mol_HumanImpactIDs"

# Define a dictionary to map Subregion values to IDs
subregion_mapping = {
    "Greater Antilles": 10,
    "Gulf of Mexico": 20,
    "Florida": 30,
    "Bahamian": 40,
    "Eastern Caribbean": 50,
    "Guianan": 60,
    "Southern Caribbean": 70,
    "Southwestern Caribbean": 80,
    "Western Caribbean": 90
}

# Add a Subregion_ID field
arcpy.management.AddField(in_feature, "Subregion_ID", "DOUBLE")

# Calculate the SubregionID field using the dictionary above
arcpy.management.CalculateField(in_feature, "Subregion_ID", "subregion_mapping.get(!Subregion!, None)", "PYTHON3")


<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\Caribbean_detailed_subregions_mol_HumanImpactIDs'>

In [3]:
# Convert the subreion layer to raster using the subregion ID as the value
# Set the cell output size as the same as the Human Impact layer cell size

in_feature = "Caribbean_detailed_subregions_mol_HumanImpactIDs"
out_feature = "\\Caribbean_detailed_subregions_mol_raster"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

with arcpy.EnvManager(cellSize="cumulative_impact_2013"):
    arcpy.conversion.FeatureToRaster(
        in_features= in_feature,
        field="Subregion_ID",
        out_raster= output_gdb + out_feature,
        cell_size=r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb\cumulative_impact_2013"
    )



In [4]:
# Assign MPAs in the study area an identification number of 100

# First make a copy of the MPA layer
in_feature = "WDPA_MPA_dis"
out_feature = "\\WDPA_MPA_dis_ID"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

arcpy.conversion.ExportFeatures(
    in_features= in_feature,
    out_features=output_gdb + out_feature,
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
)

# Add a new field called MPA_ID to the new layer
arcpy.management.AddField(out_feature, "MPA_ID", "LONG")

# Calculate the new field with a constant value of 100 for all rows
arcpy.management.CalculateField(out_feature, "MPA_ID", "100", "PYTHON3")

<Result 'F:\\Bex\\ArcGIS\\Ecological_coherence_2023\\Ecological_coherence_2023.gdb\\\\WDPA_MPA_dis_ID'>

In [5]:
# Convert and MPA layer to raster
# Set the cell output size to the same as the Human Impact layer cell size

in_feature = "WDPA_MPA_dis_ID"
out_feature = "\\WDPA_MPA_dis_ID_raster"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

with arcpy.EnvManager(cellSize="cumulative_impact_2013"):
    arcpy.conversion.FeatureToRaster(
        in_features= in_feature,
        field="MPA_ID",
        out_raster= output_gdb + out_feature,
        cell_size=r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb\cumulative_impact_2013"
    )

In [32]:
# Using raster calculator, add the raster MPA layer, the reclassified human impacts layer, and the raster subregion layer.


arcpy.env.workspace = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

# Define the input raster paths
input1 = "WDPA_MPA_dis_ID_raster.tif"
input2 = "Impact_Carib_ReclassQuart_Adq4_3.tif"
input3 = "Caribbean_detailed_subregions_mol_raster.tif"
output_gdb = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Ecological_coherence_2023.gdb"

# Specify the output raster name
output = "\\ImpactReclass_MPA_Subregions_Add_Adq4_3"

# Use RasterCalculator to add the three rasters
expression = "{} + {} + {}".format(input1, input2, input3)
output_raster = arcpy.sa.RasterCalculator([input1, input2, input3], [input1, input2, input3], expression)

output_raster.save(output_gdb + output)


In [52]:
# Export the attribute table from ImpactReclass_MPA_Subregions_Add_Adq4_3 to be a dataframe

# Specify the raster dataset name
raster_dataset = "ImpactReclass_MPA_Subregions_Add_Adq4_3"

# Use TableToNumPyArray to retrieve data from the raster attribute table
arr = arcpy.da.TableToNumPyArray(raster_dataset, "*")

# Convert the numpy array to a pandas DataFrame
df = pd.DataFrame(arr)

# Print or use the DataFrame as needed
print(df)

    OBJECTID  Value     Count
0          1    111   67601.0
1          2    112    9607.0
2          3    121   14718.0
3          4    122    5955.0
4          5    131   11882.0
5          6    132    5287.0
6          7    141   53240.0
7          8    142      96.0
8          9    151   26990.0
9         10    152  152020.0
10        11    161    2853.0
11        12    162     201.0
12        13    171   21755.0
13        14    172   15526.0
14        15    181   85903.0
15        16    182   14996.0
16        17    191   79591.0
17        18    192   11261.0


In [53]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Adequacy"
table_name = "ImpactReclass_MPA_Subregions_Add_Adq4_3.xlsx"

df.to_excel(os.path.join(table_outputs_folder, table_name))

In [54]:
# Define the subregion mapping
subregion_mapping = {
    1: "Greater Antilles",
    2 : "Gulf of Mexico",
    3: "Florida",
    4: "Bahamian",
    5: "Eastern Caribbean",
    6: "Guianan",
    7: "Southern Caribbean",
    8: "Southwestern Caribbean",
    9: "Western Caribbean"
}

# Add a new "subregion" field to the DataFrame based on the second digit of "Value"
df['subregion'] = (df['Value'] // 10) % 10
df['subregion'] = df['subregion'].map(subregion_mapping)

# Add a new field for the total number of cells per subregion
df["Total_cells"] = df.groupby("subregion")["Count"].transform("sum")

# Add a new field "total_low_impact_cells" based on the condition
df["total_low_impact_cells"] = df.loc[df["Value"] % 10 == 1, "Count"]

# Fill NaN values with 0 if necessary
df["total_low_impact_cells"].fillna(0, inplace=True)

# Add a new field "percent_low_impact_cells"
df["percent_low_impact_cells"] = (df["total_low_impact_cells"] / df["Total_cells"]) * 100

# Remove rows where the 3rd digit in the "Value" field is 2 (unneeded rows for the final output)
df = df[df["Value"] % 10 != 2]

# Reset the index after removing rows
df.reset_index(drop=True, inplace=True)

# Print or use the updated DataFrame as needed
print(df)


   OBJECTID  Value    Count               subregion  Total_cells  \
0         1    111  67601.0        Greater Antilles      77208.0   
1         3    121  14718.0          Gulf of Mexico      20673.0   
2         5    131  11882.0                 Florida      17169.0   
3         7    141  53240.0                Bahamian      53336.0   
4         9    151  26990.0       Eastern Caribbean     179010.0   
5        11    161   2853.0                 Guianan       3054.0   
6        13    171  21755.0      Southern Caribbean      37281.0   
7        15    181  85903.0  Southwestern Caribbean     100899.0   
8        17    191  79591.0       Western Caribbean      90852.0   

   total_low_impact_cells  percent_low_impact_cells  
0                 67601.0                 87.556989  
1                 14718.0                 71.194311  
2                 11882.0                 69.206127  
3                 53240.0                 99.820009  
4                 26990.0                 15.0773

In [55]:
#Export to excel

table_outputs_folder = r"F:\Bex\ArcGIS\Ecological_coherence_2023\Tables\Adequacy"
table_name = "ImpactReclass_MPA_Subregions_Add_Adq4_3_results.xlsx"

df.to_excel(os.path.join(table_outputs_folder, table_name))